In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run -m rpy2.situation

rpy2 version:
3.5.1
Python version:
3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:25:59) 
[GCC 10.3.0]
Looking for R's HOME:
    Environment variable R_HOME: None
    Calling `R RHOME`: /opt/R/mkl/4.0.3/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64:/opt/intel/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64:/opt/gcc/10.2.0/lib64:/opt/gcc/10.2.0/lib:/opt/R/mkl/4.0.3/lib/R/lib
R version:
    In the PATH: R version 4.0.3 (2020-10-10) -- "Bunny-Wunnies Freak Out"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/opt/R/mkl/4.0.3/lib/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'rt', 'dl', 'm', 'icuuc', 'icui18n']
  library_dirs:
  ['/opt/R/mkl/4.0.3/lib/R/lib']
  extra_compile_args:
  []
  extra_link_args:
  ['-Wl,--export-dynamic', '-fopenmp']


In [3]:
import numpy as np

In [4]:
np.__config__.show()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread', 'mkl_rt']
    library_dirs = ['/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl', '/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/include', '/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/lib']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread', 'mkl_rt']
    library_dirs = ['/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl', '/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/include', '/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/lib']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread', 'mkl_rt']
    library_dirs = ['/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/

In [5]:
import sys
sys.path.append('/home/saikat/Documents/work/sparse-regression/simulation/eb-linreg-dsc/dsc/functions')
import simulate

def center_and_scale(Z):
    dim = Z.ndim
    if dim == 1:
        Znew = Z / np.std(Z)
        Znew = Znew - np.mean(Znew)
    elif dim == 2:
        Znew = Z / np.std(Z, axis = 0)
        Znew = Znew - np.mean(Znew, axis = 0).reshape(1, -1)
    return Znew

def initialize_ash_prior(k, scale = 2):
    w = np.zeros(k)
    w[0] = 0.0
    w[1:(k-1)] = np.repeat((1 - w[0])/(k-1), (k - 2))
    w[k-1] = 1 - np.sum(w)
    #prior_grid = (np.arange(k) * 5) + 0.1
    #prior_grid *= scale
    sk2 = np.square((np.power(scale, np.arange(k) / k) - 1))
    prior_grid = np.sqrt(sk2)
    return w, prior_grid

n = 200
p = 500
p_causal = 20
pve = 0.7
k = 20

X, y, Xtest, ytest, btrue, strue = simulate.equicorr_predictors(n, p, p_causal, pve, rho = 0.0, seed = 10)
X      = center_and_scale(X)
Xtest  = center_and_scale(Xtest)
wk, sk = initialize_ash_prior(k, scale = 2)

In [6]:
'''
mr.ash.alpha
'''
from mrashpen.inference.mrash_wrapR import MrASHR

mrash_r = MrASHR(option = "r2py", debug = True)
mrash_r.fit(X, y, sk, binit = np.zeros(p), winit = wk, s2init = 1.0)

Mr.ASH terminated at iteration 1785.
